# CS 276 Programming Assignment 2: Spelling Corrector

## I. Overview

In this assignment, we will build a probabilistic spelling corrector to automatically correct errors in queries. More formally, given a (possibly corrupt) raw query $R$, our goal is to find the intended query $Q$ which maximizes the probability $P(Q\mid R)$. That is, we want to guess the query which the user probably meant to submit. By Bayes' Theorem we have
$$
    P(Q\mid R) = \frac{P(R\mid Q)P(Q)}{P(R)}\propto P(R\mid Q)P(Q).
$$
Since our goal is to find the value of $Q$ which maximizes $P(Q\mid R)$, this shows it is sufficient to maximize $P(R\mid Q)P(Q)$. With the above formulation in mind, we will build a probabilistic spelling corrector consisting of 4 parts:
  1. **Language Model.**
      Estimates the prior distribution of unigrams and bigrams, allowing us to estimate $P(Q)$. We will use maximum-likelihood estimation, which counts the occurrences of token unigrams and bigrams in the training corpus in order to determine their prior probabilities.
  2. **Edit Probability Model.**
      Estimates the likelihood of errors that may occur in a query, which allows us to estimate $P(R\mid Q)$. In particular, this component estimates the probability of characters being mistakenly deleted, inserted, substituted, or transposed in a query term.
  3. **Candidate Generator.**
      Takes a raw query $R$ submitted by the user, and generates candidates for $Q$.
  4. **Candidate Scorer.**
      Combines (1), (2), and (3) to compute $Q^{*} = \arg\max_{Q}P(Q\mid R)$. That is, for each $Q$ generated by the candidate generator, the scorer uses the language model to estimate $P(Q)$ and uses the edit probability model to estimate $P(R\mid Q)$, and finally chooses $Q$ which maximizes $P(Q)P(R\mid Q)$.

## II. Assignment Details

The assignment is due at **4:00 PM PST on Tuesday, May 7th, 2019**. We have split the assignment up into the following parts:
  1. [Task 1: Spelling Correction with Uniform Edit Costs](#uniform): **55%** of your total grade for this assignment depends on a correctly implemented solution for task 1. Your solution will be evaluated on a hidden test set, and full credit will be given to models that are within 1% of the staff implementation's test-set accuracy or higher. We do not publish the test set queries or our accuracy on the test set. However, as a guideline for performance, the staff implementation with uniform edit probability model gets **82.42% on the dev set.** We will give partial credit on a non-linear scale (which disproportionately favors models that are closer to our threshold for full credit, as an encouragement to squeeze out more performance improvements).
  2. [Task 2: Spelling Correction with Empirical Edit Costs](#empirical): **25%** of your total grade is based on your implementation of task 2. Full credit will be granted for accuracy levels within 1% of the staff implementation's test-set accuracy or higher. Again, we do not publish our test set accuracy, but the staff implementation with empirical edit probability model gets **87.91% on the dev set.** As with Task 1, we will give partial for lower accuracy levels, we will give partial credit on a non-linear scale, with credit accruing more rapidly as your solution gets closer to the target.
  3. [Written Report](#written): **20%** of your grade is based on the 1-2 page report that you will submit through Gradescope. See [Section VI](#written) for instructions and grading breakdown.
  4. [Extra Credit (Optional)](#extra): **Up to 10%** extra credit will be awarded for implementing extensions, with an explanation in the report. It is not necessary for the extensions to radically improve accuracy to get credit. As described in [Section VII](#extra), you can also get a small amount of extra credit if your system is a top performer in terms of accuracy or running time.

The submission procedure is the same as in PA1, but we repeat the instructions here for your reference:
  - This assignment should be done in teams of two or individually. Assignments are graded the same for one and two person teams.
  - The notebook will automatically generate Python files in `submission` folder. To submit your assignment, **upload the Python files to the PA2-code assignment on Gradescope.** Note that you need to upload all the individual files in the `submission` folder without zipping it.
  - While solving the assignment, do **NOT** change class and method names, otherwise the autograder tests will fail.
  - You'll also have to **upload a PDF version of the notebook (which would be primarily used to grade your report section of the notebook) to PA2-PDF assignment on Gradescope.** Note that directly converting the PDF truncates code cells. To get a usable PDF version, first click on `File > Print Preview`, which will open in a new tab, then print to PDF using your browser's print functionality.
  - After uploading the PDF make sure you tag all the relevant pages to each question. We reserve the right to penalize for mistagged submissions.
  - If you are solving the assignment in a team of two, add the other student as a group member after submitting the assignment. Do **NOT** submit the same assignment twice.

#### A Note on Numerical Stability

Many of the probabilities we will encounter in this assignment are very small. When we multiply many small numbers together, there is a risk of [underﬂow](https://en.wikipedia.org/wiki/Arithmetic_underflow). Therefore, it is common practice to perform this type of probability calculation in log space. Recall that:
  1. The log function is monotonically increasing, therefore $\arg\max p = \arg\max\log p$.
  2. We have $\log(pq) = \log p + \log q$, and by extension $\log\left(\prod_{i} p_i\right) = \sum_{i}\log p_i$.

As a result, if we want to maximize $P(\textbf{x}) = P(x_1)P(x_2)\cdots P(x_n)$, we can equivalently maximize $\log P(\textbf{x}) = \log P(x_1) + \log P(x_2) + \cdots + \log P(x_n)$. **For numerical stability, we recommend that you use this log-space formulation throughout the assignment.**

<a id="dataset"></a>
## III. Dataset

The dataset you will be working with for this assignment is available as a zip file at [this link](http://web.stanford.edu/class/cs276/pa/pa2-data.zip). The unzipped data directory will contain the following subdirectories:
  - **Language Modeling Morpus (`pa2-data/corpus/`).** 99,904 documents crawled from the stanford.edu domain. The corpus is organized in a block structure found at `pa2-data/corpus/`, where you'll find 10 files. Each line in a file represents the text of a single document. You will use the tokens in these documents to build a language model.
  - **Query Training Set (`pa2-data/training_set/`).** 819,722 pairs of misspelled queries and their corresponding corrected versions, with each pair separated by an edit distance of at most one. The two queries are tab-separated in the file `pa2-data/training_set/edit1s.txt`. You will use this data to build a probability model for the "noisy channel" of spelling errors.
  - **Query Dev Set (`pa2-data/dev_set`).** 455 pairs of misspelled and corrected queries, which you will use to measure the performance of your model.  There are three files in `pa2-data/dev_set/`: the (possibly) misspelled queries are in `queries.txt`, corrected versions are in `gold.txt`, and Google's suggested spelling corrections are in `google.txt`.
  
Run the following code blocks to import packages, download, and unzip the data.

In [1]:
%reload_ext autograding_magics

In [2]:
# %%tee submission/imports.py

# Import modules
import math
import os
import urllib.request
import zipfile
from collections import Counter
from tqdm import tqdm
# from numpy import argmax

<a id='uniform'></a>
## IV. Task 1: Spelling Correction with Uniform Edit Costs (55%)

### IV.1. Language Model

We will now build a language model to estimate $P(Q)$ from the training corpus. We will treat $Q$ as a sequence of terms $(w_1, \ldots, w_n)$ whose probability is computed as
$$
P(w_1, \ldots, w_n) = P(w_1)P(w_2\mid w_1)\cdots P(w_n\mid w_{n-1}),
$$
where $P(w_1)$ is the unigram probability of term $w_1$, and $P(w_{i}\mid w_{i-1})$ is the bigram probability of $(w_{i-1}, w_i)$ for $i \in \{2, \ldots, n\}$.

#### IV.1.1. Calculating Unigram and Bigram Probabilities

Our language model will use the maximum likelihood estimates (MLE) for both probabilities, which turn out to be their observed frequencies:
$$
\begin{align*}
    P_{\text{MLE}}(w_i) & = \frac{\texttt{count}(w_i)}{T},
    &
    P_{\text{MLE}}(w_i\mid w_{i-1}) & = \frac{\texttt{count}((w_{i}, w_{i-1}))}{\texttt{count}(w_{i-1})},
\end{align*}
$$
where $T$ is the total number of tokens in our corpus, and where $\texttt{count}$ simply counts occurrences of unigrams or bigrams in the corpus. In summary, computing unigram probabilities $P(w_i)$ and bigram probabilities $P(w_{i}\mid w_{i-1})$ is a simple matter of counting the unigrams and bigrams that appear throughout the corpus.

Fill out the following code block to count the unigrams and bigrams in our corpus.

In [3]:
# %%tee submission/language_model_part1.py

class LanguageModel:
    """Models prior probability of unigrams and bigrams."""

    def __init__(self, corpus_dir='pa2-data/corpus', lambda_=0.1):
        """Iterates over all whitespace-separated tokens in each file in
        `corpus_dir`, and counts the number of occurrences of each unigram and
        bigram. Also keeps track of the total number of tokens in the corpus.

        Args:
            corpus_dir (str): Path to directory containing corpus.
            lambda_ (float): Interpolation factor for smoothing by unigram-bigram
                interpolation. You only need to save `lambda_` as an attribute for now, and
                it will be used later in `LanguageModel.get_bigram_logp`. See Section
                IV.1.2. below for further explanation.
        """
        self.lambda_ = lambda_
        self.total_num_tokens = 0        # Counts total number of tokens in the corpus
        
        self.unigram_counts = {}          # Initialize dictionary to maintain unigram counts
        self.bigram_counts ={}            # Initialize dictionary to maintain bigram counts
        
        for i in range(10):
            file = corpus_dir + '/' + str(i) + '.txt'
            with open(file, 'r') as fp:
                doc = fp.read()
                doc = doc.split()
                self.total_num_tokens += len(doc)
                for tok_id in range(len(doc)):
                    try:
                        self.unigram_counts[doc[tok_id]]+=1
                    except:
                        self.unigram_counts[doc[tok_id]]=1
                    try:
                        self.bigram_counts[doc[tok_id]+ " " + doc[tok_id+1]]+=1
                    except:
                        if(tok_id!=len(doc)-1):
                            self.bigram_counts[doc[tok_id]+ " " + doc[tok_id+1]]=1

Now that we have counted the unigrams and bigrams in our corpus, we will add methods for computing query probabilities. First, however, a note about handling bigrams which never occur in our corpus:

<a id='smoothing'></a>
#### IV.1.2. Smoothing by Interpolation

The unigram probability model will also serve as our vocabulary, since we are making the assumption that our query language is derived from our document corpus. As a result, we do not need to perform [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) on our unigram probabilities, since our candidates will be drawn from this very vocabulary. However, even if we have two query terms that are both members of our query language, there is no guarantee that their corresponding *bigram* appears in our training corpus. To handle this data sparsity problem, we will *interpolate* unigram and bigram probabilities to get our ﬁnal conditional probability estimates:
$$
P(w_2\mid w_1) = \lambda P_{\text{MLE}}(w_2) + (1 - \lambda)P_{\text{MLE}}(w_2\mid w_1).
$$
Try setting $\lambda$ to a small value (say, 0.1) in the beginning, and experiment later with varying this parameter to see if you can get better correction accuracies on the development dataset. However, be careful not to overﬁt your development dataset. (You might consider reserving a small portion of your development data to tune the parameters).

Fill out the functions below to complete our `LanguageModel` class.

In [4]:
# %%tee submission/language_model_part2.py

# NOTE: Syntax on the following line just extends the `LanguageModel` class
class LanguageModel(LanguageModel):
    def get_unigram_logp(self, unigram):
        """Computes the log-probability of `unigram` under this `LanguageModel`.

        Args:
            unigram (str): Unigram for which to compute the log-probability.

        Returns:
            log_p (float): Log-probability of `unigram` under this
                `LanguageModel`.
        """
        try:
            return self.unigram_counts[unigram] / self.total_num_tokens
        except:
            return 0.000000000000000001                      

    def get_bigram_logp(self, w_1, w_2):
        """Computes the log-probability of `unigram` under this `LanguageModel`.

        Note:
            Use self.lambda_ for the unigram-bigram interpolation factor.

        Args:
            w_1 (str): First word in bigram.
            w_2 (str): Second word in bigram.

        Returns:
            log_p (float): Log-probability of `bigram` under this
                `LanguageModel`.
        """
        try: 
            return math.log(self.lambda_*self.get_unigram_logp(w_2) + (1 - self.lambda_)*(self.bigram_counts[w_1 + " " + w_2]/self.unigram_counts[w_1]), 10)

        except:
            return -18

    def get_query_logp(self, query):
        """Computes the log-probability of `query` under this `LanguageModel`.

        Args:
            query (str): Whitespace-delimited sequence of terms in the query.

        Returns:
            log_p (float): Log-probability assigned to the query under this
                `LanguageModel`.
        """
        query = query.split()
        
        # Implementing the P(w1,...wn) formula
        probability_product = 0
        for i in range(1,len(query)):
            probability_product = probability_product + self.get_bigram_logp(query[i - 1], query[i])
        probability_product = probability_product + math.log(self.get_unigram_logp(query[0]), 10)            # log(unigram) because get_unigram_logp() does not return log
        return probability_product

In [5]:
# Make sure your implementation passes the following sanity checks
# Note: Constructing the language model could take 30 seconds or longer
# We suggest using `tqdm` to track progress in your `LanguageModel.__init__` function.
lm = LanguageModel()

assert len(lm.unigram_counts) == 347071, 'Invalid num. unigrams: {}'.format(len(lm.unigram_counts))
assert len(lm.bigram_counts) == 4497257, 'Invalid num. bigrams: {}'.format(len(lm.bigram_counts))
assert lm.total_num_tokens == 25498340, 'Invalid num. tokens: {}'.format(lm.total_num_tokens)

# Test a reasonable query with and without typos (you should try your own)!
query_wo_typo = "stanford university"
query_w_typo = "stanfrod universit"

p_wo_typo = math.exp(lm.get_query_logp(query_wo_typo))                           # WHY exp???
p_w_typo = math.exp(lm.get_query_logp(query_w_typo))
print('P("{}") == {}'.format(query_wo_typo, p_wo_typo))
print('P("{}") == {}'.format(query_w_typo, p_w_typo))
if p_wo_typo <= p_w_typo:
    print('\nAre you sure "{}" should be assigned higher probability than "{}"?'
          .format(query_w_typo, query_wo_typo))
print('All tests passed!')

P("stanford university") == 0.08400910983345951
P("stanfrod universit") == 1.2497632217906412e-11
All tests passed!


### IV.2. Edit Probability Model

The edit probability model attempts to estimate $P(R\mid Q)$. That is, for a fixed candidate query $Q$, the edit probability model estimates the probability that a (possibly corrupt) raw query $R$ was submitted. We quantify the distance between the candidate query $Q$ and the actual input $R$ using the [Damerau-Levenshtein distance](https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance). In Damerau-Levenshtein distance, the possible edits are **insertion**, **deletion**, **substitution**, and **transposition**, each involving single characters as operands. We have provided a base class for `EditCostModel`s below.

In [6]:
# %%tee submission/base_edit_probability_model.py

class BaseEditProbabilityModel:
    def get_edit_logp(self, edited, original):
        """Gets the log-probability of editing `original` to arrive at `edited`.
        The `original` and `edited` arguments are both single terms that are at
        most one edit apart.
        
        Note: The order of the arguments is chosen so that it reads like an
        assignment expression:
            > edited := EDIT_FUNCTION(original)
        or, alternatively, you can think of it as a (unnormalized) conditional probability:
            > log P(edited | original)

        Args:
            edited (str): Edited term.
            original (str): Original term.

        Returns:
            logp (float): Log-probability of `edited` given `original`
                under this `EditProbabilityModel`.
                
        """
        raise NotImplementedError  # Force subclass to implement this method

**It is important to understand that `get_edit_logp` will be called with `original` and `edited` each being single terms that are at most 1 edit apart.** Moreover, its outputs need not be normalized probabilities that sum to 1 over all possible edits to `original` (you can think of the return value more as a "likelihood score" than a true probability). We provide an example usage below for clarity:
```python
epm = EditProbabilityModelSubclass(...)  # You will define such a subclass later
original = 'stanford'
edited = 'stanfrod'                      # Edited by transposing 'o' and 'r'
score = epm.get_edit_logp(edited, original)
```

#### IV.2.1. Uniform-Cost Edit Model

As a first pass, we will implement a *uniform-cost edit model.* This model simplifies the computation of the edit probability by assuming that every individual edit in the Damerau-Levenshtein distance has the same probability. You should try a range of values for your uniform edit probability, but in the beginning 0.01 - 0.10 is appropriate. One important thing to remember in building your model is that the user's input query $R$ may indeed be the right one in a majority of cases (*i.e.,* $R = Q$). Thus we typically choose a high ﬁxed probability for `edited == original`; a reasonable range is 0.90 - 0.95.

The edit probability model that you construct here will be used when you rank candidates for query corrections. The candidate generator (described in the next section) will make one edit at a time, and it will call the edit probability model each time it makes a single edit to a term, summing log-probabilities for multi-edit changes. Therefore, all you need to do in this part is to calculate the probability of `edited` given that it is **at most one edit from `original`.** This means that `get_edit_logp` will be very simple in this case.

Fill out the following class to implement a uniform-cost edit model.

In [7]:
# %%tee submission/uniform_edit_probability_model.py

class UniformEditProbabilityModel(BaseEditProbabilityModel):
    def __init__(self, edit_prob=0.05):
        """
        Args:
            edit_prob (float): Probability of a single edit occurring, where
                an edit is an insertion, deletion, substitution, or transposition,
                as defined by the Damerau-Levenshtein distance.
        """
        self.edit_prob = edit_prob

    def get_edit_logp(self, edited, original):
        """Gets the log-probability of editing `original` to arrive at `edited`.
        The `original` and `edited` arguments are both single terms that are at
        most one edit apart.
        
        Note: The order of the arguments is chosen so that it reads like an
        assignment expression:
            > edited := EDIT_FUNCTION(original)
        or, alternatively, you can think of it as a (unnormalized) conditional probability:
            > log P(edited | original)

        Args:
            edited (str): Edited term.
            original (str): Original term.

        Returns:
            logp (float): Log-probability of `edited` given `original`
                under this `EditProbabilityModel`.
        """
        prob = 0.0
        if edited == original:
            prob = 1 - 0.4 # Fixed probablity
        else:
            prob = 0.4                                  
        return math.log(prob, 10)

In [8]:
EDIT_PROB = 0.4
epm = UniformEditProbabilityModel(edit_prob=EDIT_PROB)
edited, original = 'did you go to stanford on university at stranforde', 'did you go to stranford on unversit at stranforde'
epm.get_edit_logp(edited, original)

-0.39794000867203755

In [9]:
math.log(0.8,10)            # -0.0223
#math.log(0.05, 10)           # -1.301

# -12*-0.096
lm.get_bigram_logp("stranford", "unviersity") + math.log(lm.get_unigram_logp("stranford"), 10)

# -2.4*-0.69
lm.get_bigram_logp("stanford", "university") + math.log(lm.get_unigram_logp("stanford"), 10)

#cs.get_score("stranford unviersity", epm.get_edit_logp("stranford unviersity", "stranford unviersity"))

-2.4768300356208153

Make sure you pass the following sanity checks:

In [10]:
EDIT_PROB = 0.4
epm = UniformEditProbabilityModel(edit_prob=EDIT_PROB)

# Test a basic edit
edited, original = 'stanfrod', 'stanford'
assert math.isclose(epm.get_edit_logp(edited, original), math.log(EDIT_PROB, 10))

# Test a non-edit
assert math.isclose(epm.get_edit_logp(original, original), math.log(1. - EDIT_PROB, 10))

print('All tests passed!')

All tests passed!


### IV.3. Candidate Generator

Recall that the candidate generator takes a raw query $R$ submitted by the user, and generates candidates for the intended query $Q$. Since we know that more than 97% of spelling errors are found within an edit distance of 2 from the user's intended query, we encourage you to consider possible query corrections that are within distance 2 of $R$. This is the approach taken by Peter Norvig in [his essay on spelling correction](http://norvig.com/spell-correct.html). However, it is not tractable to use a pure "brute force" generator that produces all possible strings within distance 2 of $R$, because for any $R$ of non-trivial length, the number of candidates would be enormous. Thus we would have to evaluate the language and edit probability models on a huge number of candidates.


#### IV.3.1. Candidate Generator with Restricted Search Space

We can make the naïve approach tractable by aggressively narrowing down the search space while generating candidates. There are many valid approaches to efficient candidate generation, but here are a few basic ideas:
  - Begin by looking at *each individual term* in the query string $R$, and consider all possible edits that are distance 1 from that term.
  - Remember that you might consider hyphens and/or spaces as elements of your character set. This will allow you to consider some relatively common errors, like when a space is accidentally inserted in a word, or two terms in the query were mistakenly separated by a space when they should actually be joined.
  - Each time you generate an edit to a term, make sure that the edited term appears in the dictionary. (Remember that we have assumed that all words in a valid candidate query will be found in our training corpus, as mentioned above in [Section IV.1.2](#smoothing) above).
  - If you have generated possible edits to multiple individual terms, take the Cartesian product over these terms to produce a complete candidate query that includes edits to multiple terms. (But remember that you probably shouldn't go beyond a total edit distance of 2 for the query overall).
  
Again, there are many possible extensions and variations on the strategies mentioned here. We encourage you to explore some diﬀerent options, and then describe in your written report the strategies that you ultimately used, and how you optimized their performance. Note that **solutions that exhaustively generate and score all possible query candidates at edit distances 1 and 2 will run too slowly and will not receive full credit.**

In [15]:
# %%tee submission/candidate_generator.py

# CHANGES IN THIS VERSION - COMMENTS MADE AT LINE #73, #87

class CandidateGenerator:
    # Alphabet to use for insertion and substitution
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
                'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
                '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                ' ', ',', '.', '-']

    def __init__(self, lm, epm):
        """
        Args:
            lm (LanguageModel): Language model to use for prior probabilities, P(Q).
            epm (EditProbabilityModel): Edit probability model to use for P(R|Q).
        """
        self.lm = lm
        self.epm = epm
        self.vocab = set(lm.unigram_counts.keys())

    def get_num_oov(self, query):
        """Get the number of out-of-vocabulary (OOV) words in `query`."""
        return sum(1 for w in query.strip().split()
                   if w not in self.lm.unigram_counts)

    def filter_and_yield(self, query, lp):
        if query.strip() and self.get_num_oov(query) == 0:
            yield query, lp
            
    def in_vocab(self, words):
        return set(word for word in words if word in self.vocab)
    
    def edit_distance_one(self, word):
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in self.alphabet]
        inserts    = [L + c + R               for L, R in splits for c in self.alphabet]
        
        return set(deletes + transposes + replaces + inserts)
    
    def edit_distance_two(self, word):
        return set(e2 for e1 in self.edit_distance_one(word) for e2 in self.edit_distance_one(e1))
    
    def get_candidates(self, query):
        """Starts from `query`, and performs EDITS OF DISTANCE <=2 to get new
        candidate queries. To make scoring tractable, only returns/yields
        candidates that satisfy certain criteria (ideas for such criteria are
        described in bullet points above).

        Hint: We suggest you implement a helper function that takes a term and
            generates all possible edits of distance one from that term.
            It should probably only return edits that are in the vocabulary
            (i.e., edits for which `self.get_num_oov(edited) == 0`).

        Args:
            query (str): Starting query.

        Returns:
            Iterable over tuples (cdt, cdt_edit_logp) of candidates and
                their associated edit log-probabilities. Return value could be
                a list or a generator yielding tuples of this form.
        """
        
        terms = query.strip().split()                   # List of terms in the query
        
        distance_one = []                               # Stores one edit distance terms  [[candidate terms,...], [index of word in query]]
        distance_two = []                               # Stores two edit distance terms  [[candidate terms,...], [index of word in query]]
        
        pos = 0
        
#         It's a list. Dictionary -> Repeated words gives errors. There are better ways of doing it, but lists are simple
        terms_dict = []                            
        
        for i in range(len(terms)): 
            terms_dict.append([terms[i], i])
        
        for key, value in terms_dict:
            temp = self.edit_distance_one(key)
                
            distance_one.append([temp, value])
            distance_two.append([self.edit_distance_two(key).difference(temp), value])     # OPTIMIZATION 1: 'difference' to avoid duplicates

            
            
#       very_berry_temp->doesn't matter
#       Basically, the 2nd try and except for both accepted1 and accepted2 makes sure we add the position,
#       even if there are NO edited terms present in the vocab. In such a case, add the query term itself
#       as the value.

        # OPTIMIZATION 2 : Remove one edited terms not in vocab
        accepted1 = {}                                 # Stores accepted 1-edit distance terms. {index:{terms,}}
        for termsEdited, index in distance_one:
            for j in termsEdited:
                if j in self.vocab:
                    try:
                        accepted1[index].add(j)
                    except:
                        accepted1[index] = {j,}
            try: 
                very_berry_temp = accepted1[index]
            except:
                accepted1[index] = {terms[index], }
                        
        accepted2 = {}                                # Stores accepted 2-edit distance terms. {index:{terms,}}
        for termsEdited, index in distance_two:
            sampledSet = itertools.islice(termsEdited, 3)
            for j in termsEdited:
                if j in self.vocab:
                    try:
                        accepted2[index].add(j)
                    except:
                        accepted2[index] = {j,}
            try: 
                very_berry_temp = accepted2[index]
            except:
                accepted2[index] = {terms[index], }
        
        # Generate Candidate Queries with one-edit and zero-edit distance replacements
        terms_indexed = [[k, v] for k,v in terms_dict]                      # {current_word : index_in_query}
        query_terms = terms
        candidate_queries_1 = []                                                    # Final candidate query list of one-edit replacements
        cq = []                                                                     # Temporary list of candidate queries
        candidate = ""                                                              # Temporary candidate
        for i in range(len(terms_indexed)-1):
            i_word, i_index = terms_indexed[i][0], terms_indexed[i][1]
            candidate = ' '.join(query_terms[:i_index])                      # i = consider i'th to_be_edited word to be the first replacement
                                                                             # Candidate includes all words upto the i'th word as-is.
            for edited_word in accepted1[i_index]:
                if candidate:                                                # if to handle correct addition of whitespaces
                    cq.append(candidate + " " + edited_word)
                else:
                    cq.append(edited_word)
                                                                             # cq holds all possible queries with correction on the i'th term (and upto ith)
                                                                             # Next step: Generate all possible queries hereforth
            if candidate:                                                    # Include un-edited current word as well
                cq.append(candidate + " " + i_word)
            else:
                cq.append(i_word)
                
            j = i+1
            candidate = ""
            
            cq2 = []
            for ind in cq: 
                cq2.append(ind + " " + ' '.join(query_terms[i_index + 1:]))
            candidate_queries_1 += cq2                                   # Add completed queries to the final list
            
            while(j<len(terms_indexed)):
                j_word, j_index = terms_indexed[j][0], terms_indexed[j][1]   # Next Incorrect word and Index of the next incorrect word
                candidate = ' '.join(query_terms[i_index+1:j_index])         # All the correct words in between the prev incorrect and current incorrect
                cq2 = []                                                     # Temporary Candidate Query List
                for edited_word in accepted1[j_index]:
                    for ind in range(len(cq)):                               # Append correct words in between + edited terms to the half-candidate queries and complete
                        if candidate:
                            cq2.append(cq[ind] + " " + candidate + " " + edited_word + " " + ' '.join(query_terms[j_index+1:]))
                        else:
                            cq2.append(cq[ind] + " " + edited_word + " " + ' '.join(query_terms[j_index+1:]))
                j+=1                                                         # Next incorrect word
                candidate_queries_1 += cq2                                   # Add completed queries to the final list
                
            cq = []
            
        '''
        print("\n---------------------ONE EDIT DISTANCE--------------------------\n")
        print(candidate_queries_1)
        print("\n---------------------                 --------------------------\n")
        '''
                
        # Generate Candidate Queries with a Single two-edit replacement
        
        pos = 0
        candidate_queries_2 = []
        candidate = ""
        query_terms = terms
        for term, value in terms_dict:
            for i in accepted2[pos]:
                candidate_queries_2.append(candidate + i + " " + ' '.join(query_terms[pos+1:]))
            candidate += term + " "                                     # Exclude correction of current incorrect term and append as-is.
            pos += 1
        '''
        print("\n---------------------TWO EDIT DISTANCE--------------------------\n")
        print(candidate_queries_2)
        print("\n---------------------                 --------------------------\n")
        '''
                
        
        # Adding my doubts here:
        '''
        1. Candidate generation has to be done for each term - but how will the cartesian product work?
        2. How to ensure cartesian product terms have edit distance <= 2? 
        3. Once we get the valid candidate queries, epms for each word have to be summed or multiplied?
        4. All the above steps still seems extremely computationally expensive? How do you optimize it?
        '''
        
        # Yield the unedited query first
        # We provide this line as an example of how to use `self.filter_and_yield`
        candidate = candidate_queries_1 + candidate_queries_2
        res= []
        for edited_query in candidate: 
            #yield from self.filter_and_yield(query, self.epm.get_edit_logp(edited_query, query))
            res.append([edited_query.strip(), self.epm.get_edit_logp(edited_query.strip(), query)])
            
        #res.remove([query, math.log(0.8, 10)]) # TOREMOVE ORIGINAL QUERY FROM LIST. SHOULD NOT BE THERE
        #print(res)
        return res
        
        
model = CandidateGenerator(LanguageModel(), UniformEditProbabilityModel(BaseEditProbabilityModel))
#model.get_candidates("did you go to stranford on unversit at stranforde")
#model.get_candidates('stranford unviersity')

In [16]:
testcg = CandidateGenerator(lm, epm)
testcg.get_candidates("pagge 1 page 2 page")

[['page 1 page 2 page', -0.39794000867203755],
 ['paige 1 page 2 page', -0.39794000867203755],
 ['paggi 1 page 2 page', -0.39794000867203755],
 ['bagge 1 page 2 page', -0.39794000867203755],
 ['hagge 1 page 2 page', -0.39794000867203755],
 ['pogge 1 page 2 page', -0.39794000867203755],
 ['pagge 1 page 2 page', -0.22184874961635637],
 ['page u page 2 page', -0.39794000867203755],
 ['paige u page 2 page', -0.39794000867203755],
 ['paggi u page 2 page', -0.39794000867203755],
 ['bagge u page 2 page', -0.39794000867203755],
 ['hagge u page 2 page', -0.39794000867203755],
 ['pogge u page 2 page', -0.39794000867203755],
 ['pagge u page 2 page', -0.39794000867203755],
 ['page i1 page 2 page', -0.39794000867203755],
 ['paige i1 page 2 page', -0.39794000867203755],
 ['paggi i1 page 2 page', -0.39794000867203755],
 ['bagge i1 page 2 page', -0.39794000867203755],
 ['hagge i1 page 2 page', -0.39794000867203755],
 ['pogge i1 page 2 page', -0.39794000867203755],
 ['pagge i1 page 2 page', -0.39794000

Make sure your candidate generator passes the following sanity checks. Feel free to add more tests here as you see fit.

In [17]:
cg = CandidateGenerator(lm, epm)
query = 'stanford university'
num_candidates = 0
did_generate_original = False
for candidate, candidate_logp in cg.get_candidates(query):
    num_candidates += 1
    if candidate == query:
        did_generate_original = True

    assert cg.get_num_oov(query) == 0, \
        "You should not generate queries with out-of-vocab terms ('{}' has OOV terms)".format(candidate)

assert 1e2 <= num_candidates <= 1e4, \
    "You should generate between 100 and 10,000 terms (generated {})".format(num_candidates)

assert did_generate_original, "You should generate the original query ({})".format(query)

### Begin your code

### End your code

print('All tests passed!')

All tests passed!


### IV.4. Candidate Scorer

The candidate scorer's job is to find the most likely query $Q$ given the raw query $R$. It does this by combining the language model for $P(Q)$, the edit probability model for $P(R\mid Q)$, and the candidate generator (to get candidates for $Q$). Formally, given raw query $R$, the candidate scorer outputs
$$
    Q^{*} = \arg\max_{Q_{i}} P(Q_{i}\mid R) = \arg\max_{Q_{i}} P(R\mid Q_{i}) P(Q_{i}),
$$
where the max is taken over candidate queries $Q_{i}\in\{Q_1, \ldots, Q_{n}\}$ produced by the candidate generator given $R$.

#### IV.4.1. Candidate Scorer with Weighting
When combining probabilities from the language model and the edit probability model, we can use a parameter to weight the two models differently:
$$
    P(Q\mid R)\propto P(R\mid Q)P(Q)^{\mu}.
$$
Start out with $\mu = 1$, and then experiment later with different values of $\mu$ to see which one gives you the best spelling correction accuracy. Again, be careful not to overfit your development dataset. 

Fill out the following class to complete the spelling corrector with uniform edit cost model.

In [18]:
# %%tee submission/candidate_scorer.py

class CandidateScorer:
    """Combines the `LanguageModel`, `EditProbabilityModel`, and
    `CandidateGenerator` to produce the most likely query Q given a raw query R.
    Since the candidate generator already uses the edit probability model, we
    do not need to take the edit probability model as an argument in the constructor.
    """
    def __init__(self, lm, cg, mu=1.):
        """
        Args:
            lm (LanguageModel): Language model for estimating P(Q).
            cg (CandidateGenerator): Candidate generator for generating possible Q.
            mu (float): Weighting factor for the language model (see write-up).
                Remember that our probability computations are done in log-space.
        """
        self.lm = lm
        self.cg = cg
        self.mu = mu
    
    def get_score(self, query, log_edit_prob):
        """Uses the language model and `log_edit_prob` to compute the final
    b    score for a candidate `query`. Uses `mu` as weighting exponent for P(Q).

        Args:
            query (str): Candidate query.
            log_edit_prob (float): Log-probability of candidate query given
                original query (i.e., log(P(R|Q), where R is `query`).

        Returns:
            log_p (float): Final score for the query, i.e., the log-probability
                of the query.
        """
        ### Begin your code
        
        p_q = self.lm.get_query_logp(query)
        try:
            return log_edit_prob*p_q
        except:
            return -100 # Why are we returning 100 here?

        ### End your code

    def correct_spelling(self, r):
        """Corrects spelling of raw query `r` to get the intended query `q`.

        Args:
            r (str): Raw input query from the user.

        Returns:
            q (str): Spell-corrected query. That is, the query that maximizes
                P(R|Q)*P(Q) under the language model and edit probability model,
                restricted to Q's generated by the candidate generator.
        """
        ### Begin your code
        
        # generate candidate queries
        candidates = self.cg.get_candidates(r) # get candidates here using self.cg
        final_scores = [0]*len(candidates)
#         for i in candidates:
#             final_scores.append(0)
            
        min_index = 0
        for i in range(len(final_scores)):
            final_scores[i] = self.get_score(candidates[i][0],candidates[i][1])
            if(final_scores[i]<final_scores[min_index]):
                min_index = i
        '''        
        for i in range(len(final_scores)):
            print(candidates[i][0], candidates[i][1], final_scores[i])
        print("\n#######################################################################################################\n")
        print(candidates[min_index][0], "\t", final_scores[min_index])
        '''
        return candidates[min_index][0]

        ### End your code

In [19]:
# Assumes LanguageModel lm was already built above
print('Building edit probability model...')
epm = UniformEditProbabilityModel()
print('Building candidate generator...')
cg = CandidateGenerator(lm, epm)
print('Building candidate scorer model...')
cs = CandidateScorer(lm, cg, mu=1.0)
print('Running spelling corrector...')

# Add your own queries here to test your spelling corrector
queries = [('stranford unviersity', 'stanford university'),
             ('stanford unviersity', 'stanford university'),
             ('sanford university', 'stanford university')]
for query, expected in queries:
    corrected = cs.correct_spelling(query)
    print("\t'{}' corrected to '{}'".format(query, corrected))
    assert corrected == expected, "Expected '{}', got '{}'".format(expected, corrected)
print('All tests passed!')

Building edit probability model...
Building candidate generator...
Building candidate scorer model...
Running spelling corrector...
	'stranford unviersity' corrected to 'stanford university'
	'stanford unviersity' corrected to 'stanford university'
	'sanford university' corrected to 'stanford university'
All tests passed!


#### IV.4.2. Dev Set Evaluation (Uniform)

Now that we have constructed a basic spelling corrector, we will evaluate its performance on the held-out dev set. Recall that the dev set is stored across the files in `pa2-data/dev_set/`:
  - `queries.txt`: One raw query $R$ per line.
  - `google.txt`: Google's corrected queries $Q$ (one per line, same order as `queries.txt`).
  - `gold.txt`: Ground-truth queries $Q$ (again, one per line, same order).
  
Run the following cells to evaluate your spelling corrector on the dev set using your uniform edit probability model. We will also evaluate your model on a private test set after submission. For full credit, your spelling corrector with uniform edit probability model should achieve accuracy within 1% of the staff implementation *on the test set.* **We do not provide test set queries, but as a guideline for performance, the staff implementation gets 82.42% accuracy on the dev set.**

In [20]:
def dev_eval(candidate_scorer, verbose=False):
    """Evaluate `candidate_scorer` on the dev set."""
    query_num = 1
    yours_correct = 0
    google_correct = 0
    # Read originals, ground-truths, Google's predictions
    dev_dir = 'pa2-data/dev_set/'
    with tqdm(total=455, unit=' queries') as pbar, \
            open(os.path.join(dev_dir, 'queries.txt'), 'r') as query_fh, \
            open(os.path.join(dev_dir, 'gold.txt'), 'r') as gold_fh, \
            open(os.path.join(dev_dir, 'google.txt'), 'r') as google_fh:
        while True:
            # Read one line
            query = query_fh.readline().rstrip('\n')
            print("Query = ", query)
            if not query:
                # Finished all queries
                break
            corrected = candidate_scorer.correct_spelling(query)
            corrected = ' '.join(corrected.split())  # Squash multiple spaces
            gold = gold_fh.readline().rstrip('\n')
            google = google_fh.readline().rstrip('\n')

            # Count whether correct
            if corrected == gold:
                yours_correct += 1
            if google == gold:
                google_correct += 1

            # Print running stats
            yours_accuracy = yours_correct / query_num * 100
            google_accuracy = google_correct / query_num * 100
            if verbose:
                print('QUERY {:03d}'.format(query_num))
                print('---------')
                print('(original):      {}'.format(query))
                print('(corrected):     {}'.format(corrected))
                print('(google):        {}'.format(google))
                print('(gold):          {}'.format(gold))
                print('Google accuracy: {}/{} ({:5.2f}%)\n'
                      .format(google_correct, query_num, google_accuracy))
                print('Your accuracy:   {}/{} ({:5.2f}%)'
                      .format(yours_correct, query_num, yours_accuracy))
            
            pbar.set_postfix(google='{:5.2f}%'.format(google_accuracy),
                             yours='{:5.2f}%'.format(yours_accuracy))
            pbar.update()
            query_num += 1

In [77]:
# Set verbose=True for debugging output
# For reference, our implementation takes ~1 min, 40 sec to run and gets 82.42% accuracy
dev_eval(cs, verbose=False)

  0%|                                                                                                                             | 0/455 [00:00<?, ? queries/s]

Query =  quade quad cache xontroller
Query =  ['quade', 'quad', 'cache', 'xontroller']
I =  0
I =  1
I =  2
I =  3
dict_keys([0, 1, 2, 3])


  0%|▏                                                                                     | 1/455 [00:00<06:29,  1.17 queries/s, google=100.00%, yours=100.00%]

Query =  co2 in
Query =  ['co2', 'in']
I =  0
I =  1
dict_keys([0, 1])


  0%|▍                                                                                       | 2/455 [00:01<05:01,  1.50 queries/s, google=50.00%, yours=50.00%]

...

Query =  jim pleased our
Query =  ['jim', 'pleased', 'our']
I =  0
I =  1
I =  2
dict_keys([0, 1, 2])


100%|█████████████████████████████████████████████████████████████████████████████████████▊| 454/455 [10:43<00:01,  1.24s/ queries, google=83.04%, yours=70.26%]

Query =  to skin strain changes
Query =  ['to', 'skin', 'strain', 'changes']
I =  0
I =  1
I =  2
I =  3
dict_keys([0, 1, 2, 3])


100%|██████████████████████████████████████████████████████████████████████████████████████| 455/455 [10:43<00:00,  1.06s/ queries, google=83.08%, yours=70.33%]

Query =  


<a id='empirical'></a>
## V. Task 2: Spelling Correction with Empirical Edit Costs (25%)


### V.1. Improved Edit Probability Model

Now that our spelling corrector is working correctly with a basic edit probability model, we will turn our attention to a somewhat more realistic approach to edit probabilities. In this task, we will learn these edit probabilities from the empirical error data provided in `data/training_set/edit1s.txt`.

#### V.1.1. Empirical Edit Costs

As outlined in [Section III](#dataset) above, you have been given a list of query pairs that are precisely edit distance 1 from each other. The ﬁrst step for this task is to devise a simple algorithm to determine which speciﬁc edit exists between the two queries in each pair. By aggregating the counts of all such edits over all queries, you can estimate the probability of each individual edit. The edit probability calculation is described in more detail in the [lecture handout on spelling correction](http://web.stanford.edu/class/cs276/handouts/spell_correction.pdf). As an example, if you need to determine the probability of the letter 'e' being (mistakenly) replaced by the letter 'a' in a query, you should calculate:
$$
    P(\texttt{sub}[a, e]) = \frac{\texttt{count}(\texttt{sub}[a, e])}{\texttt{count}(e)}.
$$
Note that the insertion and deletion operator probabilities are conditioned on the character before the character being operated on &mdash; which also means that you should devise an appropriate solution to handle the special case of insertions or deletions occurring at the beginning of a word. Finally, to account for the inevitable problem of data sparsity in our edit training ﬁle, you should apply Laplace add-one smoothing to the edit probabilities, as described in the lecture handout (linked above).

In [ ]:
%%tee submission/empirical_edit_probability_model.py

class Edit:
    """Represents a single edit in Damerau-Levenshtein distance.
    We use this class to count occurrences of different edits in the training data.
    """
    INSERTION = 1
    DELETION = 2
    TRANSPOSITION = 3
    SUBSTITUTION = 4

    def __init__(self, edit_type, c1=None, c2=None):
        """
        Members:
            edit_type (int): One of Edit.{NO_EDIT,INSERTION,DELETION,
                TRANSPOSITION,SUBSTITUTION}.
            c1 (str): First (in original) char involved in the edit.
            c2 (str): Second (in original) char involved in the edit.
        """
        self.edit_type = edit_type
        self.c1 = c1
        self.c2 = c2


class EmpiricalEditProbabilityModel(BaseEditProbabilityModel):

    START_CHAR = ''      # Used to indicate start-of-query
    NO_EDIT_PROB = 0.92  # Hyperparameter for probability assigned to no-edit

    def __init__(self, training_set_path='pa2-data/training_set/edit1s.txt'):
        """Builds the necessary data structures to compute log-probabilities of
        distance-1 edits in constant time. In particular, counts the unigrams
        (single characters), bigrams (of 2 characters), alphabet size, and
        edit count for insertions, deletions, substitutions, and transpositions.

        Hint: Use the `Edit` class above. It may be easier to write the `get_edit`
        function first, since you should call that function here.

        Note: We suggest using tqdm with the size of the training set (819722) to track
        the initializers progress when parsing the training set file.

        Args:
            training_set_path (str): Path to training set of empirical error data.
        """
        # Your code needs to initialize all four of these data structures
        self.unigram_counts = Counter()  # Maps chars c1 -> count(c1)
        self.bigram_counts = Counter()   # Maps tuples (c1, c2) -> count((c1, c2))
        self.alphabet_size = 0           # Counts all possible characters

        # Maps edit-types -> dict mapping tuples (c1, c2) -> count(edit[c1, c2])
        # Example usage: 
        #   > e = Edit(Edit.SUBSTITUTION, 'a', 'b')
        #   > edit_count = self.edit_counts[e.edit_type][(e.c1, e.c2)]
        self.edit_counts = {edit_type: Counter()
                            for edit_type in (Edit.INSERTION, Edit.DELETION,
                                              Edit.SUBSTITUTION, Edit.TRANSPOSITION)}

        with open(training_set_path, 'r') as training_set:
            for example in tqdm(training_set, total=819722):
                edited, original = example.strip().split('\t')

                ### Begin your code

                ### End your code

    def get_edit(self, edited, original):
        """Gets an `Edit` object describing the type of edit performed on `original`
        to produce `edited`.

        Note: Only edits with an edit distance of at most 1 are valid inputs.

        Args:
            edited (str): Raw query, which contains exactly one edit from `original`.
            original (str): True query. Want to find the edit which turns this into `edited`.

        Returns:
            edit (Edit): `Edit` object representing the edit to apply to `original` to get `edited`.
                If `edited == original`, returns None.
        """
        ### Begin your code

        ### End your code

    def get_edit_logp(self, edited, original):
        """Gets the log-probability of editing `original` to arrive at `edited`.
        The `original` and `edited` arguments are both single terms that are at
        most one edit apart.
        
        Note: The order of the arguments is chosen so that it reads like an
        assignment expression:
            > edited := EDIT_FUNCTION(original)
        or, alternatively, you can think of it as a (unnormalized) conditional probability:
            > log P(edited | original)

        Args:
            edited (str): Edited term.
            original (str): Original term.

        Returns:
            logp (float): Log-probability of `edited` given `original`
                under this `EditProbabilityModel`.
        """
        ### Begin your code

        ### End your code

Run the following cells to evaluate your spelling corrector on the dev set using your empirical edit probability model. We will also evaluate your model on a private test set after submission. For full credit, your spelling corrector with uniform edit probability model should achieve accuracy within 1% of the staff implementation *on the test set.* **We do not provide test set queries, but as a guideline for performance, the staff implementation gets 87.91% accuracy on the dev set.**

In [ ]:
# Build spelling corrector for evaluation on the dev set
# For reference, our initialization times are 25 sec for lm, and 1 min, 40 sec for epm
lm = LanguageModel()
epm = EmpiricalEditProbabilityModel()
cg = CandidateGenerator(lm, epm)
cs = CandidateScorer(lm, cg, mu=1.0)

In [ ]:
# Set verbose=True for debugging output
# For reference our implementation takes ~2 min, 30 sec to run and gets 87.91% accuracy
dev_eval(cs, verbose=False)

<a id='written'></a>
## VI. Written Report (20%)

Be sure to document any design decisions you made, and give some brief rationale for them. Please keep your report concise.

#### VI.1. Overall System Design (5%)

Provide a concise (at most 5 sentences) description of the overall system design.

  > Your Answer Here

#### VI.2. Smoothing and Related Techniques (5%)

Give a short analysis of smoothing techniques used in this assignment. For example, you might produce a plot comparing different values for $\lambda$ in unigram-bigram interpolation.

  > Your Answer Here

#### VI.3. Optimizations for Candidate Generation (5%)

Provide a brief description of the techniques you used for optimizing candidate generation. Be sure to include an analysis of the amount by which each optimization sped up the overall spelling correction system, as well as any changes in accuracy you were able to measure.

  > Your Answer Here

#### VI.4. Tuning Parameters (5%)
Provide at least two plots showing how accuracy varies as you change parameter values (*e.g.,* $\mu$ and $\lambda$). Comment briefly (1-2 sentences) on each plot.

  > Your Answer Here

<a id='extra'></a>
## VII. Extra Credit (Optional, up to 10%)


We have listed a few ideas here, but really any extensions that go above and beyond the scope of tasks 1 and 2 will be considered.

1. **Expanded edit model.** We saw (or will see) in lecture that there are sometimes spelling errors that may not be within a "naive" edit distance 2 of the correct phrase, but that may have a conceptual basis that makes them very common and understandable. (Substituting 'ph' for 'f', or vice versa, is one such example.) Can you incorporate these types of errors into the edit probabilities of your edit probability model?
2. **Empirical edit costs using Wikipedia.** In task 2, you used the dataset of queries 1 edit distance apart to learn edit probabilities. If you look at the queries in this dataset, you will observe that most of these queries are related to the Stanford corpus, the same corpus used to build the language model. It would be interesting to explore what happens if the channel model and language model are learned from diﬀerent datasets (and hence diﬀerent distributions of the underlying data). To this end, you can use a dataset of spelling errors collected from Wikipedia and available on Peter Norvig’s website (http://norvig.com/ngrams/spell-errors.txt).
3. **Alternate Smoothing.** Try other smoothing algorithms (such as Kneser-Ney smoothing) to better capture probabilities in the training corpus.
4. **K-gram index.** To deal with unseen words, it is possible to develop a measure for the probability of that word being spelled correctly by developing a character k-gram index over your corpus. For example, a q not followed by a u should lead to a low probability. This index can also help you generate candidate corrections much more eﬃciently.
5. **Levenshtein Automata.** You can do even faster candidate generation using a Levenshtein transducer (http://en.wikipedia.org/wiki/Levenshtein_transducer), which uses a ﬁnite state automata for fuzzy matching of words. There is an experimental implementation in Python at https://gist.github.com/491973, but it needs to be generalized to perform the transposition operation too. This tutorial might be helpful: http://blog.notdot.net/2010/07/Damn-Cool-Algorithms-Levenshtein-Automata.

Finally, we will give a small amount of extra credit to the best spell correction systems, measured in terms of both accuracy and running time (as computed on our hidden test data). The top 5 systems according to either metric will receive 5% each, while the next 15 systems will receive 2.5% each.

**If you decide to tackle an extra credit option, give a brief description of your approach and results below.**

  > Your Answer Here